In [1]:
import pandas as pd
import requests
import sqlite3

from collections import defaultdict
from pathlib import Path

In [2]:
import spacy
from spacy import displacy

# nlp = spacy.load("pt_core_news_lg")
nlp = spacy.load(Path('model'))
ner = nlp.get_pipe('ner')

In [3]:
con = sqlite3.connect("requerimentos.db")

In [4]:
df = pd.read_sql("SELECT * FROM requerimento", con)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1677 entries, 0 to 1676
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   requerimento      1677 non-null   object 
 1   ano               1677 non-null   int64  
 2   numero            1677 non-null   int64  
 3   ementa            1677 non-null   object 
 4   autores           1677 non-null   object 
 5   id                1677 non-null   int64  
 6   numero_protocolo  1675 non-null   float64
 7   texto_original    1671 non-null   object 
dtypes: float64(1), int64(3), object(4)
memory usage: 104.9+ KB


In [6]:
df.head()

,requerimento,ano,numero,ementa,autores,id,numero_protocolo,texto_original
0,Requerimento de Vereador(a) nº 1 de 2021,2021,1,"Requerimento n° 01/2021, do Vereador Odemir Ja...",13,2664,48.0,https://sapl.santoantoniodaplatina.pr.leg.br/m...
1,Requerimento de Vereador(a) nº 2 de 2021,2021,2,"Requerimento n° 02/2021, do Vereador Odemir Ja...",13,2665,49.0,https://sapl.santoantoniodaplatina.pr.leg.br/m...
2,Requerimento de Vereador(a) nº 3 de 2021,2021,3,"Requerimento n° 03/2021, do Vereador Odemir Ja...",13,2666,50.0,https://sapl.santoantoniodaplatina.pr.leg.br/m...
3,Requerimento de Vereador(a) nº 4 de 2021,2021,4,"Solicita ao Executivo que proceda a ""operação ...",142,2670,62.0,https://sapl.santoantoniodaplatina.pr.leg.br/m...
4,Requerimento de Vereador(a) nº 5 de 2021,2021,5,Solicita ao Executivo pavimentação asfáltica n...,14,2671,63.0,https://sapl.santoantoniodaplatina.pr.leg.br/m...


In [7]:
def get_entities(text):
    doc = nlp(text)
    ents = defaultdict(list)
    for ent in doc.ents:
        ents[ent.label_].append(ent.text)
        
    return ents

In [8]:
tmp = df['ementa'].apply(get_entities)

df = pd.concat([df, tmp.apply(pd.Series)], axis=1)

tmp[:5]

0    {'DOC': ['Requerimento n°'], 'OCCUPATION': ['V...
1    {'DOC': ['Requerimento n°'], 'OCCUPATION': ['V...
2    {'DOC': ['Requerimento n°'], 'OCCUPATION': ['V...
3    {'ORG': ['Executivo'], 'REGION': ['Jardim Mont...
4    {'ORG': ['Executivo'], 'STREET': ['Rua William...
Name: ementa, dtype: object

In [9]:
columns = ['OCCUPATION', 'PERSON', 'REGION', 'ORG', 'STREET', 'MISC', 'DOC', 'DATE', 'LOC']

df[columns] = df[columns].applymap(lambda x : tuple(x) if type(x) == list else x)

In [10]:
df.head()

,requerimento,ano,numero,ementa,autores,id,numero_protocolo,texto_original,DOC,OCCUPATION,PERSON,REGION,ORG,STREET,MISC,LOC,DATE
0,Requerimento de Vereador(a) nº 1 de 2021,2021,1,"Requerimento n° 01/2021, do Vereador Odemir Ja...",13,2664,48.0,https://sapl.santoantoniodaplatina.pr.leg.br/m...,"(Requerimento n°,)","(Vereador Odemir Jacob, Secretário Estadual de...","(Sr. Carlos Alberto Gebrim Preto,)","(Santo Antônio da Platina,)",NaN,NaN,NaN,NaN,NaN
1,Requerimento de Vereador(a) nº 2 de 2021,2021,2,"Requerimento n° 02/2021, do Vereador Odemir Ja...",13,2665,49.0,https://sapl.santoantoniodaplatina.pr.leg.br/m...,"(Requerimento n°,)","(Vereador Odemir Jacob, Deputado Estadual)","(Mauro Rafael Moraes e Silva,)","(Santo Antônio da Platina,)",NaN,NaN,NaN,NaN,NaN
2,Requerimento de Vereador(a) nº 3 de 2021,2021,3,"Requerimento n° 03/2021, do Vereador Odemir Ja...",13,2666,50.0,https://sapl.santoantoniodaplatina.pr.leg.br/m...,"(Requerimento n°,)","(Vereador Odemir Jacob, Presidente da FUNDEPAR)","(Sr. Alessandro Oliveira,)",NaN,"(Colégio Estadual Rio Branco,)",NaN,NaN,NaN,NaN
3,Requerimento de Vereador(a) nº 4 de 2021,2021,4,"Solicita ao Executivo que proceda a ""operação ...",142,2670,62.0,https://sapl.santoantoniodaplatina.pr.leg.br/m...,NaN,NaN,NaN,"(Jardim Monte das Oliveiras,)","(Executivo,)",NaN,NaN,NaN,NaN
4,Requerimento de Vereador(a) nº 5 de 2021,2021,5,Solicita ao Executivo pavimentação asfáltica n...,14,2671,63.0,https://sapl.santoantoniodaplatina.pr.leg.br/m...,NaN,NaN,NaN,"(Jardim Monte das Oliveiras, Bairro Santo Antô...","(Executivo,)","(Rua William Vilas Boas,)",NaN,NaN,NaN


In [11]:
df_street = df.loc[~df['STREET'].isna()].explode('STREET')

In [12]:
df_street.sample(5)

,requerimento,ano,numero,ementa,autores,id,numero_protocolo,texto_original,DOC,OCCUPATION,PERSON,REGION,ORG,STREET,MISC,LOC,DATE
107,Requerimento de Vereador(a) nº 108 de 2021,2021,108,Requer que seja expedido ofício ao Excelentíss...,14,2786,315.0,https://sapl.santoantoniodaplatina.pr.leg.br/m...,NaN,"(Prefeito Municipal,)","(Senhor José da Silva Coelho Neto,)",NaN,"(Departamento Municipal de Trânsito, Escola Mu...",Rua Elsa Soboleski,NaN,NaN,NaN
439,Requerimento de Vereador(a) nº 440 de 2021,2021,440,"Requer ao Excelentíssimo Prefeito Municipal, S...",148,3155,1246.0,https://sapl.santoantoniodaplatina.pr.leg.br/m...,NaN,"(Prefeito Municipal,)","(Senhor José da Silva Coelho Neto,)","(Jardim São João,)","(Narley Velasco Gonçalves,)",Rua Amazonas,NaN,NaN,NaN
545,Requerimento de Vereador(a) nº 546 de 2021,2021,546,"Requer ao Excelentíssimo Prefeito Municipal, S...",15,3270,1497.0,https://sapl.santoantoniodaplatina.pr.leg.br/m...,NaN,"(Prefeito Municipal,)","(Senhor José da Silva Coelho Neto,)","(Jardim Santo André,)","(Departamento Municipal de Trânsito,)",Rua Manoel Pimenta Brito,NaN,NaN,NaN
13,Requerimento de Vereador(a) nº 14 de 2021,2021,14,Solicita ao Executivo uma faixa elevada na Rua...,20,2683,74.0,https://sapl.santoantoniodaplatina.pr.leg.br/m...,NaN,NaN,NaN,NaN,"(Executivo,)",Rua Dep.,NaN,NaN,NaN
138,Requerimento de Vereador(a) nº 139 de 2021,2021,139,Requer que seja expedido ofício ao Excelentíss...,142,2821,380.0,https://sapl.santoantoniodaplatina.pr.leg.br/m...,NaN,"(Prefeito Municipal,)","(Senhor José da Silva Coelho Neto, Sr. Tiega, ...",NaN,"(Departamento Municipal de Trânsito,)",Rua dos Estudantes,NaN,NaN,NaN


In [13]:
df_street.info()

<class 'pandas.core.frame.DataFrame'>
Index: 618 entries, 4 to 1674
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   requerimento      618 non-null    object 
 1   ano               618 non-null    int64  
 2   numero            618 non-null    int64  
 3   ementa            618 non-null    object 
 4   autores           618 non-null    object 
 5   id                618 non-null    int64  
 6   numero_protocolo  616 non-null    float64
 7   texto_original    618 non-null    object 
 8   DOC               30 non-null     object 
 9   OCCUPATION        247 non-null    object 
 10  PERSON            231 non-null    object 
 11  REGION            348 non-null    object 
 12  ORG               541 non-null    object 
 13  STREET            618 non-null    object 
 14  MISC              27 non-null     object 
 15  LOC               49 non-null     object 
 16  DATE              11 non-null     object 
dtypes

In [15]:
df_street.columns

Index(['requerimento', 'ano', 'numero', 'ementa', 'autores', 'id',
       'numero_protocolo', 'texto_original', 'DOC', 'OCCUPATION', 'PERSON',
       'REGION', 'ORG', 'STREET', 'MISC', 'LOC', 'DATE'],
      dtype='object')

In [14]:
df_street.to_csv('2023-06-10-requerimento-ruas.csv', index=False)